In [1]:
%%capture
!wget https://files.grouplens.org/datasets/movielens/ml-25m.zip;

In [2]:
%%capture
!mkdir zips

In [3]:
%%capture
!mv ml-25m.zip zips

In [4]:
%%capture
!mkdir data

In [5]:
%%capture
!unzip zips/ml-25m.zip -d data

In [6]:
%%capture
!mv data/ml-25m/* data/ && rmdir data/ml-25m

In [7]:
%%capture
!pip install pyspark

In [1]:
import pyspark
from pyspark.sql import SparkSession  
spark = SparkSession.builder.appName('movie-lens').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/17 12:22:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
movies = spark.read.load("./data/movies.csv", format='csv', header = True)
ratings = spark.read.load('./data/ratings.csv', format='csv', header = True)
links = spark.read.load("./data/links.csv", format='csv', header = True)
tags = spark.read.load("./data/tags.csv", format='csv', header = True)
movies.show()
ratings.show()
links.show()
tags.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [3]:
movie = movies.select("movieId", "title", "genres")
movie.printSchema()
rating = ratings.select("userId", "movieId", "rating")
rating.printSchema()
link = links.select("movieId", "imdbId", "tmdbId")
link.printSchema()
tag = tags.select("userId", "movieId", "tag")
tag.printSchema()

root
 |-- movieId: string (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)

root
 |-- movieId: string (nullable = true)
 |-- imdbId: string (nullable = true)
 |-- tmdbId: string (nullable = true)

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- tag: string (nullable = true)



24/07/17 12:23:00 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
movies_df = movies.withColumn("movieId", movies["movieId"].cast("int"))
movies_df.printSchema()
ratings_df = ratings.withColumn('userId', ratings['userId'].cast('int')).\
withColumn('movieId', ratings['movieId'].cast('int')).withColumn('rating', ratings['rating'].cast('float'))
ratings_df.printSchema()
links_df = links.withColumn("movieId", links["movieId"].cast("int")).\
withColumn("imdbId", links["imdbId"].cast("int")).\
withColumn("tmdbId", links["tmdbId"].cast("int"))
links_df.printSchema()
tags_df = tags.withColumn("userId", tags["userId"].cast("int")).\
withColumn("movieId", tags["movieId"].cast("int"))
tags_df.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: float (nullable = true)
 |-- timestamp: string (nullable = true)

root
 |-- movieId: integer (nullable = true)
 |-- imdbId: integer (nullable = true)
 |-- tmdbId: integer (nullable = true)

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- tag: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [5]:
train_data = {}
validation_data = {}
test_data = {}
data_frames = [("movies", movies_df), ("ratings", ratings_df), ("links", links_df), ("tags", tags_df)]
for (name, frame) in data_frames:
    train, validation, test = frame.randomSplit([0.6,0.2,0.2], seed = 0)
    train_data[name] = train
    validation_data[name] = validation
    test_data[name] = test

for (name, data) in train_data.items():
    print(f"Number of values for {name} for training is: ", data.count())
for (name, data) in validation_data.items():
    print(f"Number of values for {name} for validation is: ", data.count())
for (name, data) in test_data.items():
    print(f"Number of values for {name} for testing is: ", data.count())

Number of values for movies for training is:  37458


Number of values for ratings for training is:  14998964
Number of values for links for training is:  37458


Number of values for tags for training is:  656055
Number of values for movies for validation is:  12443


Number of values for ratings for validation is:  5000767
Number of values for links for validation is:  12443


Number of values for tags for validation is:  218637
Number of values for movies for testing is:  12522


Number of values for ratings for testing is:  5000364
Number of values for links for testing is:  12522
Number of values for tags for testing is:  218668
